<a href="https://colab.research.google.com/github/rehanbchinoy/Math-156-project/blob/main/dubliners_remake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

import base64
import requests

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#read the txt file to str
master = "https://raw.githubusercontent.com/rehanbchinoy/Math-156-project/main/dubliners_james_joyce.txt"
req = requests.get(master)
dubliners = req.text

In [4]:
len(dubliners)

377403

In [5]:
#unique characters in the file
vocab = sorted(set(dubliners))
print(f'{len(vocab)} unique characters')

82 unique characters


## Process the Text

### Vectorize

In [6]:
#function that converts character tokens to numeric ids
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [7]:
#function that converts numeric ids to character tokens
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [8]:
#function that combines character tokens back to strings
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Creating training examples and targets

In [9]:
#convert dubliners from strings to numeric vector
all_ids = ids_from_chars(tf.strings.unicode_split(dubliners, 'UTF-8'))
all_ids

<tf.Tensor: shape=(377403,), dtype=int64, numpy=array([39, 27, 24, ..., 47, 50, 10])>

In [10]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [11]:
#verify the ids dataset is correct
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
H
E
 
S
I
S
T
E
R
S









T
h
e
r
e
 
w
a
s
 
n
o
 
h
o
p
e
 
f
o
r
 
h
i
m
 
t
h
i
s
 
t
i
m
e
:
 
i
t
 
w
a
s
 
t
h
e
 
t
h
i
r
d
 
s
t
r
o
k
e
.
 
N
i
g
h
t



a
f
t
e
r
 
n
i
g
h
t
 
I
 


In [12]:
#split the text into sequences with length 100
seq_length = 100
examples_per_epoch = len(dubliners)//(seq_length+1)

In [13]:
#create the sequences based on sequences with length 101
#(it's 101 bc we need to split it into training and target sequences)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
#verify the sequences are created correctly
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'H' b'E' b' ' b'S' b'I' b'S' b'T' b'E' b'R' b'S' b'\r' b'\n' b'\r'
 b'\n' b'\r' b'\n' b'T' b'h' b'e' b'r' b'e' b' ' b'w' b'a' b's' b' ' b'n'
 b'o' b' ' b'h' b'o' b'p' b'e' b' ' b'f' b'o' b'r' b' ' b'h' b'i' b'm'
 b' ' b't' b'h' b'i' b's' b' ' b't' b'i' b'm' b'e' b':' b' ' b'i' b't'
 b' ' b'w' b'a' b's' b' ' b't' b'h' b'e' b' ' b't' b'h' b'i' b'r' b'd'
 b' ' b's' b't' b'r' b'o' b'k' b'e' b'.' b' ' b'N' b'i' b'g' b'h' b't'
 b'\r' b'\n' b'a' b'f' b't' b'e' b'r' b' ' b'n' b'i' b'g' b'h' b't' b' '
 b'I' b' ' b'h'], shape=(101,), dtype=string)


In [15]:
#generate the target sequence for each sequence
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [16]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

### Create Training Batches

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [19]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
 
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [20]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try The Model

Run the model to see if the output is as expected

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 83) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  21248     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  85075     
                                                                 
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [24]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ntaining a subject in the\r\nthird person and a predicate in the past tense. He never gave alms to\r\nbe'

Next Char Predictions:
 b'1MLTAHxEed&TZw8VM-tF951jUnX\xe2\x80\x9cf.d\roGc1fkH&FlBMMado\xc3\xa7NHsW.t\xe2\x80\x9c.bFJ__t\xc5\x93IcE\xc3\xa6sX\xe2\x80\x94\rW[UNK]WWAFoi1WdgOeL\xc3\xa8\xe2\x80\x94rHDDB\nclpeW'


## Train The Model

### Add optimizer and loss

In [25]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [26]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 83)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.4218397


Check to make sure the loss is roughly the same as the vocabulary size

In [27]:
tf.exp(mean_loss).numpy()

83.2493

In [28]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks'
# Name of the checkpoint files
#checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True)

### Execute the training

In [30]:
EPOCHS = 20

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
58/58 [==============================] - 11s 143ms/step - loss: 3.3540
Epoch 2/20
58/58 [==============================] - 9s 140ms/step - loss: 2.4043
Epoch 3/20
58/58 [==============================] - 9s 142ms/step - loss: 2.2208
Epoch 4/20
58/58 [==============================] - 9s 140ms/step - loss: 2.0803
Epoch 5/20
58/58 [==============================] - 9s 142ms/step - loss: 1.9454
Epoch 6/20
58/58 [==============================] - 9s 142ms/step - loss: 1.8225
Epoch 7/20
58/58 [==============================] - 9s 141ms/step - loss: 1.7081
Epoch 8/20
58/58 [==============================] - 9s 142ms/step - loss: 1.6070
Epoch 9/20
58/58 [==============================] - 9s 142ms/step - loss: 1.5213
Epoch 10/20
58/58 [==============================] - 9s 143ms/step - loss: 1.4505
Epoch 11/20
58/58 [==============================] - 9s 142ms/step - loss: 1.3876
Epoch 12/20
58/58 [==============================] - 9s 142ms/step - loss: 1.3278
Epoch 13/20
58/58 [=====

In [32]:
#model.save_weights('/content/drive/MyDrive/Colab Notebooks/checkpoint_epoch20')

In [33]:
model.save('/content/drive/MyDrive/Colab Notebooks/dubliners_saved_model_epoch20')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/dubliners_saved_model_epoch20/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/dubliners_saved_model_epoch20/assets


## Generate Text

In [34]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [36]:
start = time.time()
states = None
next_char = tf.constant(['Duke'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Duke pulling knee. To express through the respectfull of
Sinceralt from Bridge they used to puff your took let his nose in by
downstairs upoticular whose latesting for their little old father involdst by her
hat into easisty. Little Browne, both began to money on to it in to
Dellanch world. Saturday taking them the Pewspaper hold
hoce, it she do so?” asked Gabriel, “I see you down and So,”

Mr Browne took handing sofficely and saiding out of the night in
Stitizal attention the day a distro had been roof down at his plotes and entored the closely
in the age of his own lival pings! Miss Healy said selled upon his
moustache. He was a plain home my sons as she did not question that his
mind. Gro barrged him in her eyes and looked as if he was in go to plundave ;iphatius. O’H
assumat at herr she got one of the
other two men who were to sing that it wasn’t a joy to that was one night where once cake
their enesn by their turs and loose from the great pleasane in his creed,
stood to 

________

References:
https://arxiv.org/pdf/1308.0850.pdf
